In [1]:
# Здесь надо отформатировать и поменять все 
# Нужен пакет из одного из заданий когда excel устанавливался через conda <- его нужно было раскоментировать

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install lxml

! pip install BeautifulSoup4
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

! pip install geocoder
import geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
Wiki_Manchester = requests.get("https://en.wikipedia.org/wiki/Category:Areas_of_Manchester").text
soup = BeautifulSoup(Wiki_Manchester, 'html.parser')
Wiki_Manchester_List = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    Wiki_Manchester_List.append(row.text)
Manchester_df = pd.DataFrame({"Areas of Manchester": Wiki_Manchester_List})
Manchester_df.head(10)

,Areas of Manchester
0,Abbey Hey
1,Alport Town
2,Ancoats
3,Ardwick
4,Ardwick Green
5,Ashton Hurst (ward)
6,Ashton St. Michael's (ward)
7,Ashton Waterloo (ward)
8,Audenshaw (ward)
9,Baguley


In [3]:
print ("No. of Areas of Manchester = ", Manchester_df.shape[0])

No. of Areas of Manchester =  84


In [4]:
def get_latitude_longitude(area):
    # initialize variable to None
    latitude_longitude_coordinates = None
    # loop until get the coordinates
    while(latitude_longitude_coordinates is None):
        g = geocoder.arcgis('{}, Manchester, UK'.format(area))
        latitude_longitude_coordinates = g.latlng
    return latitude_longitude_coordinates


In [5]:
# call the function to get the coordinates, store in a new list using list comprehension
Man_areas_coordinates = [ get_latitude_longitude(area) for area in Manchester_df["Areas of Manchester"].tolist() ]
Man_areas_coordinates

[[53.46867000000003, -2.156069999999943],
 [53.44917809606844, -2.2704934421781653],
 [53.48535000000004, -2.229199999999935],
 [53.46932000000004, -2.2226199999999494],
 [53.471925922005795, -2.224585576122144],
 [53.44778977077495, -2.2425545178583004],
 [53.51808304269245, -2.5101763361676346],
 [53.51351122391121, -2.2235986233223834],
 [53.47124000000008, -2.1291799999999625],
 [53.39909000000006, -2.285609999999963],
 [53.42216517268773, -2.245972579737383],
 [42.9558586733971, -71.45901854736869],
 [53.38173000000006, -2.2612499999999613],
 [53.47839000000005, -2.2003199999999765],
 [53.51661000000007, -2.208099999999945],
 [53.47839000000005, -2.2003199999999765],
 [53.514233999921956, -2.211628499722893],
 [53.47959000000003, -2.248739999999941],
 [53.42136000000005, -2.2172799999999597],
 [53.42136000000005, -2.2172799999999597],
 [53.47626000000008, -2.2554699999999457],
 [53.49164227100497, -2.24074532773305],
 [53.44251000000003, -2.276559999999961],
 [53.43262000000004, -

In [6]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_Man_areas_coordinates = pd.DataFrame(Man_areas_coordinates, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
Manchester_df['Latitude'] = df_Man_areas_coordinates['Latitude']
Manchester_df['Longitude'] = df_Man_areas_coordinates['Longitude']
# check the neighborhoods and the coordinates
print(Manchester_df.shape)
Manchester_df

(84, 3)


,Areas of Manchester,Latitude,Longitude
0,Abbey Hey,53.468670,-2.156070
1,Alport Town,53.449178,-2.270493
2,Ancoats,53.485350,-2.229200
3,Ardwick,53.469320,-2.222620
4,Ardwick Green,53.471926,-2.224586
...,...,...,...
79,West Gorton,53.461110,-2.173060
80,"Whalley Range, Manchester",53.453020,-2.263920
81,Withington,53.434610,-2.227860
82,Woodhouse Park,53.463107,-2.166657


In [7]:
Manchester_df.to_csv("Manchester_df.csv", index=False)

In [8]:
address = 'Manchester, UK'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manchester, UK {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manchester, UK 53.4794892, -2.2451148.


In [9]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, area in zip(Manchester_df['Latitude'], Manchester_df['Longitude'], Manchester_df['Areas of Manchester']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
map_kl

In [10]:
# define Foursquare Credentials and Version
CLIENT_ID = 'DGNST2XBRTVYF0V3OVAT3KQC1ENK1YDHZ3EDTCLI2GOJMLGP' # your Foursquare ID
CLIENT_SECRET = 'OQHKOOIVWSO3IDLWWIWRX2IDFNVCCDF2ZNUGZLY1QV1CE4XF' # your Foursquare Secret
VERSION = '201901111' # Foursquare API version
radius = 2000
LIMIT = 100

venues = []
for lat, lng, area in zip(Manchester_df['Latitude'], Manchester_df['Longitude'], Manchester_df['Areas of Manchester']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".\
        format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((area, lat, lng, venue['venue']['name'], venue['venue']['location']['lat'], venue['venue']['location']['lng'], venue['venue']['categories'][0]['name']))

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Areas', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()       
        

(5215, 7)


,Areas,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abbey Hey,53.46867,-2.15607,Debdale Park,53.458288,-2.160558,Park
1,Abbey Hey,53.46867,-2.15607,The Gym Manchester Ashton Old Road,53.473077,-2.164137,Gym / Fitness Center
2,Abbey Hey,53.46867,-2.15607,PureGym,53.459749,-2.163222,Gym / Fitness Center
3,Abbey Hey,53.46867,-2.15607,Lidl,53.472882,-2.165574,Supermarket
4,Abbey Hey,53.46867,-2.15607,London Irish Pub,53.474080,-2.148228,Pub


In [11]:
venues_df.groupby(["VenueCategory"]).count()

,Areas,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude
VenueCategory,,,,,,
Afghan Restaurant,4,4,4,4,4,4
Airport,1,1,1,1,1,1
Airport Lounge,3,3,3,3,3,3
Airport Service,2,2,2,2,2,2
Airport Terminal,1,1,1,1,1,1
...,...,...,...,...,...,...
Warehouse Store,8,8,8,8,8,8
Weight Loss Center,1,1,1,1,1,1
Wine Bar,33,33,33,33,33,33


In [12]:
venues_df.groupby(["Areas"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Areas,,,,,,
Abbey Hey,39,39,39,39,39,39
Alport Town,87,87,87,87,87,87
Ancoats,100,100,100,100,100,100
Ardwick,100,100,100,100,100,100
Ardwick Green,100,100,100,100,100,100
...,...,...,...,...,...,...
West Gorton,40,40,40,40,40,40
"Whalley Range, Manchester",89,89,89,89,89,89
Withington,100,100,100,100,100,100


In [13]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 221 uniques categories.


In [14]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:224]

array(['Park', 'Gym / Fitness Center', 'Supermarket', 'Pub',
       'General College & University', 'Farmers Market', 'Discount Store',
       'Sandwich Place', 'Pizza Place', 'Grocery Store',
       'Fast Food Restaurant', 'Soccer Stadium', 'Soccer Field', 'Market',
       'Indian Restaurant', 'Italian Restaurant', 'Golf Course',
       'Tram Station', 'Train Station', 'Intersection',
       'Department Store', 'Bar', 'Deli / Bodega', 'Gastropub',
       'Cocktail Bar', 'Tapas Restaurant', 'Japanese Restaurant',
       'Tea Room', 'Coffee Shop', 'Museum', 'Food & Drink Shop',
       'Turkish Restaurant', 'Liquor Store', 'Falafel Restaurant',
       'Spanish Restaurant', 'Café', 'Nature Preserve', 'Cricket Ground',
       'Fish Market', 'Thai Restaurant', 'Concert Hall',
       'Portuguese Restaurant', 'Restaurant', 'Hotel',
       'Electronics Store', 'Gym', 'Fish & Chips Shop', 'Disc Golf',
       'Track', 'Brewery', 'Pet Store', 'Food Truck', 'Bus Line',
       'Brazilian Restaurant

In [15]:
# one hot encoding
Manchester_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
Manchester_onehot['Areas'] = venues_df['Areas'] 
# move neighborhood column to the first column
fixed_columns = [Manchester_onehot.columns[-1]] + list(Manchester_onehot.columns[:-1])
Manchester_onehot = Manchester_onehot[fixed_columns]
print(Manchester_onehot.shape)
Manchester_onehot.head()

(5215, 222)


,Areas,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
Manchester_grouped = Manchester_onehot.groupby(['Areas']).mean().reset_index()
print(Manchester_grouped.shape)
Manchester_grouped

(84, 222)


,Areas,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,Abbey Hey,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.076923,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1,Alport Town,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.022989,0.011494,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0
2,Ancoats,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.02,...,0.000000,0.000000,0.010000,0.0,0.00,0.0,0.0,0.01,0.0,0.0
3,Ardwick,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.01,...,0.000000,0.010000,0.010000,0.0,0.00,0.0,0.0,0.00,0.0,0.0
4,Ardwick Green,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.02,0.01,...,0.000000,0.000000,0.010000,0.0,0.00,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,West Gorton,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0
80,"Whalley Range, Manchester",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.011236,0.011236,0.011236,0.0,0.00,0.0,0.0,0.00,0.0,0.0
81,Withington,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.00,...,0.010000,0.000000,0.020000,0.0,0.00,0.0,0.0,0.01,0.0,0.0
82,Woodhouse Park,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.029412,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0


In [17]:
len(Manchester_grouped[Manchester_grouped["Pharmacy"] > 0])

32

In [18]:
Manchester_Pharmacy = Manchester_grouped[["Areas","Pharmacy"]]
Manchester_Pharmacy.head(40)

,Areas,Pharmacy
0,Abbey Hey,0.000000
1,Alport Town,0.000000
2,Ancoats,0.000000
3,Ardwick,0.000000
4,Ardwick Green,0.000000
5,Ashton Hurst (ward),0.000000
6,Ashton St. Michael's (ward),0.000000
7,Ashton Waterloo (ward),0.048780
8,Audenshaw (ward),0.022727
9,Baguley,0.000000


In [19]:
# set number of clusters
Cs = 3
Kmean_clustering = Manchester_Pharmacy.drop(["Areas"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=Cs, random_state=0).fit(Kmean_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [20]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
merged_dataset = Manchester_Pharmacy.copy()

# add clustering labels
merged_dataset["Cluster Labels"] = kmeans.labels_

In [21]:
merged_dataset.rename(columns={"Areas": "Area"}, inplace=True)
merged_dataset.head(40)

,Area,Pharmacy,Cluster Labels
0,Abbey Hey,0.000000,0
1,Alport Town,0.000000,0
2,Ancoats,0.000000,0
3,Ardwick,0.000000,0
4,Ardwick Green,0.000000,0
5,Ashton Hurst (ward),0.000000,0
6,Ashton St. Michael's (ward),0.000000,0
7,Ashton Waterloo (ward),0.048780,1
8,Audenshaw (ward),0.022727,0
9,Baguley,0.000000,0


In [22]:
# merge and add latitude/longitude for each area
merged_dataset = merged_dataset.join(Manchester_df.set_index('Areas of Manchester'), on='Area')

print(merged_dataset.shape)
merged_dataset.head()

(84, 5)


,Area,Pharmacy,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.0,0,53.468670,-2.156070
1,Alport Town,0.0,0,53.449178,-2.270493
2,Ancoats,0.0,0,53.485350,-2.229200
3,Ardwick,0.0,0,53.469320,-2.222620
4,Ardwick Green,0.0,0,53.471926,-2.224586


In [23]:
# sort the results by Cluster Labels
print(merged_dataset.shape)
merged_dataset.sort_values(["Cluster Labels"], inplace=True)
merged_dataset

(84, 5)


,Area,Pharmacy,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.000000,0,53.468670,-2.156070
58,New Islington,0.000000,0,53.482300,-2.221190
56,Mossley (ward),0.000000,0,53.519869,-2.038954
55,Moss Side,0.000000,0,53.457460,-2.252370
54,Miles Platting,0.000000,0,53.495350,-2.211990
...,...,...,...,...,...
15,"Bradford, Manchester",0.076923,1,53.514234,-2.211628
37,"Glenbrook, Greater Manchester",0.052632,1,53.534965,-2.240070
31,Denton South (ward),0.043478,1,53.456010,-2.113680
60,Newall Green,0.041667,1,53.384160,-2.283810


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Cs)
ys = [i+x+(i*x)**2 for i in range(Cs)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_dataset['Latitude'], merged_dataset['Longitude'], merged_dataset['Area'], merged_dataset['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [28]:
merged_dataset.loc[merged_dataset['Cluster Labels'] == 0]

,Area,Pharmacy,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.000000,0,53.468670,-2.156070
58,New Islington,0.000000,0,53.482300,-2.221190
56,Mossley (ward),0.000000,0,53.519869,-2.038954
55,Moss Side,0.000000,0,53.457460,-2.252370
54,Miles Platting,0.000000,0,53.495350,-2.211990
...,...,...,...,...,...
34,Didsbury East (ward),0.010000,0,53.405200,-2.221760
29,Crumpsall,0.027027,0,53.518890,-2.248740
21,"Cheetham, Manchester",0.000000,0,53.491642,-2.240745
27,"Clayton, Manchester",0.000000,0,53.481096,-2.167594


In [29]:
merged_dataset.loc[merged_dataset['Cluster Labels'] == 1]

,Area,Pharmacy,Cluster Labels,Latitude,Longitude
57,"Moston, Manchester",0.066667,1,53.513381,-2.207517
42,Harpurhey,0.058824,1,53.512610,-2.213980
76,Stalybridge South (ward),0.047619,1,53.461902,-2.107010
32,Denton West (ward),0.043478,1,53.456010,-2.113680
75,Stalybridge North (ward),0.041667,1,53.510161,-2.405010
47,Hyde Werneth (ward),0.041667,1,53.447764,-2.111613
7,Ashton Waterloo (ward),0.048780,1,53.513511,-2.223599
30,Denton North East (ward),0.043478,1,53.456010,-2.113680
69,"Ringway, Manchester",0.044118,1,53.362051,-2.266700
14,Blackley,0.071429,1,53.516610,-2.208100


In [30]:
merged_dataset.loc[merged_dataset['Cluster Labels'] == 2]

,Area,Pharmacy,Cluster Labels,Latitude,Longitude
50,Longdendale (ward),0.142857,2,53.46667,-2.0
